In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.metrics.cluster import contingency_matrix

In [3]:
from scipy import sparse as sp

In [4]:
import sys, os, time, fnmatch, itertools, re
from math import log

In [5]:
# https://stackoverflow.com/questions/2186525/use-a-glob-to-find-files-recursively-in-python
def get_fnames(dirname, glob='*.tree'):
    matches = []
    for root, dirnames, filenames in os.walk(dirname):
        for filename in fnmatch.filter(filenames, glob):
            matches.append(os.path.join(root, filename))
    return matches

In [6]:
fnames_treefiles = get_fnames('data/trimmed_years/infomap_out/')
fnames_treefiles = pd.DataFrame(fnames_treefiles, columns=['fpath'])
fnames_treefiles['year'] = fnames_treefiles.fpath.apply(lambda x: re.search(r'-(\d\d\d\d)\.', x).group(1)).astype(int)
fnames_treefiles = fnames_treefiles.sort_values('year').set_index('year').fpath

In [7]:
def get_tree_df(fname):

    with open(fname, 'r') as f:
        rows = []
        for line in f:
            if line[0] == "#":
                continue
            
            line = line.strip().split(' ')
            pid = int(line[2].strip('"'))
            cl = line[0]
            rank = float(line[1])
            
            rows.append( (pid, rank, cl) )
    df = pd.DataFrame(rows, columns=['pid', 'paper_rank', 'cl'])
    df['cl_bottom'] = df.cl.apply(lambda x: ':'.join(x.split(':')[:-1])).astype('category')
#     df['cl_top'] = df.cl.apply(lambda x: x.split(':')[0]).astype('category')
    
    df = df.sort_values('pid')

    return df

In [8]:
# pointwise mutual information
# https://stackoverflow.com/questions/35850582/can-pandas-dataframe-efficiently-calculate-pmi-pointwise-mutual-information
def pmi_from_df(dff, x, y):
    df = dff.copy()
    df['f_x'] = df.groupby(x)[x].transform('count')
    df['f_y'] = df.groupby(y)[y].transform('count')
    df['f_xy'] = df.groupby([x, y])[x].transform('count')
    df['pmi'] = np.log(len(df.index) * df['f_xy'] / (df['f_x'] * df['f_y']) )
    return df

In [9]:
def _get_cl_level(x, level=2):
    spl = x.split(':')[:-1]
    if level == -1:
        # special case for level == -1
        return ':'.join(spl)
    if len(spl) < level:
        return 'NA'
    else:
        return ':'.join(spl[:level])

In [10]:

def get_contingency_matrix(left, right):
    # left and right are arrays of cluster assignments (the 'codes' for pandas categories)
    return contingency_matrix(left, right, sparse=True)

def get_assignments(df, level):
    x = df.cl.apply(_get_cl_level, level=level)
    x = x.astype('category')
    return x


def get_category_name_from_code(s, code):
    return s.cat.categories[code]


In [11]:
def all_pmis_from_contingency(contingency, normalized=False):
    contingency_sum = contingency.sum()
    pi = np.ravel(contingency.sum(axis=1))
    pj = np.ravel(contingency.sum(axis=0))
    contingency_nzx, contingency_nzy, contingency_nz_val = sp.find(contingency)
    pi_diag = sp.diags(pi)
    pj_diag = sp.diags(pj)
    m = np.dot(pi_diag, contingency)
    m = np.dot(m, pj_diag)
    m = m.power(-1)
    m = m.multiply(contingency)
    m = m * contingency_sum
    mnzx, mnzy, mnz_val = sp.find(m)
    mnz_val = np.log(mnz_val)
    if normalized:
        mnz_val = mnz_val / (-np.log(contingency_nz_val/float(contingency_sum)))
    return np.array(zip(mnzx, mnzy, mnz_val), dtype=[('x_idx', int), ('y_idx', int), ('pmi', np.float)])

In [12]:
def get_all_pmis_one_level(df_left, df_right, level_left=-1, level_right=-1, normalized=True):
    # get the pmis for a (bottom level) cluster on the left against the clusters of <level> on the right
    
    left = get_assignments(df_left, level=level_left)
    left_codes = left.cat.codes
    right = get_assignments(df_right, level=level_right)
    right_codes = right.cat.codes
    contingency = get_contingency_matrix(left_codes, right_codes)
    
    pmis = all_pmis_from_contingency(contingency, normalized=normalized)
    return pmis, left, right

In [13]:
def find_max_depth(df_list):
    max_depth = 0
    for df in df_list:
        depth = df.cl_bottom.apply(lambda x: len(x.split(':')))
        this_max_depth = depth.max()
        if this_max_depth > max_depth:
            max_depth = this_max_depth
    return max_depth

def align_two_dfs(df1, df2, colname='pid'):
    df1 = df1[df1[colname].isin(df2[colname])]
    df2 = df2[df2[colname].isin(df1[colname])]
    return df1, df2

from collections import defaultdict

def get_all_levels_data(left_df, right_df, max_depth=None):
    # may take some time
    
    if max_depth is None:
        max_depth = find_max_depth([left_df, right_df])
    
    # make both dfs contain the same papers
    left_df, right_df = align_two_dfs(left_df, right_df)
    
    levels = range(1,max_depth)
    data = defaultdict(dict)
    for level_left, level_right in itertools.product(levels, repeat=2):
        this_data = get_all_pmis_one_level(left_df, right_df, level_left, level_right)
        data[level_left][level_right] = this_data
    return data


In [14]:
def data_to_df(data):
    dfs_pmi = []
    for level_left in data:
        for level_right in data[level_left]:
            this_data = data[level_left][level_right]
            this_df_pmi = pd.DataFrame(this_data[0])
            this_df_pmi['level_left'] = level_left
            this_df_pmi['level_right'] = level_right
#             this_df_pmi['cl_left'] = this_df_pmi['x_idx'].apply(lambda x: get_category_name_from_code(this_data[1], x))
#             this_df_pmi['cl_right'] = this_df_pmi['y_idx'].apply(lambda x: get_category_name_from_code(this_data[2], x))
            this_df_pmi['cl_left'] = this_df_pmi['x_idx'].map(pd.Series(this_data[1].cat.categories))
            this_df_pmi['cl_right'] = this_df_pmi['y_idx'].map(pd.Series(this_data[2].cat.categories))
            dfs_pmi.append(this_df_pmi)
    return pd.concat(dfs_pmi)

In [15]:
def get_all_levels_df(left_df, right_df, max_depth=None, return_data=True):
    data = get_all_levels_data(left_df, right_df, max_depth=max_depth)
    df = data_to_df(data)
    if return_data is True:
        return df, data
    else:
        return df

In [16]:
# def _get_cl_name(row, data, side='right'):
#     level_left = row['level_left']
#     level_right = row['level_right']
#     if side == 'right':
#         idx = int(row['y_idx'])
#         side_idx = 2
#     elif side == 'left':
#         idx = int(row['x_idx'])
#         side_idx = 1
#     return get_category_name_from_code(data[level_left][level_right][side_idx], idx)


In [17]:
years = [1980, 1990]
# fnames = fnames_treefiles.loc[1980:1970].values
fnames = [fnames_treefiles.loc[x] for x in years]
print(fnames)
dfs = []
for fname in fnames:
    df = get_tree_df(fname)
    dfs.append(df)

['data/trimmed_years/infomap_out/jstor_trimmed-1980.tree', 'data/trimmed_years/infomap_out/jstor_trimmed-1990.tree']


In [18]:
start = time.time()
df_data, data = get_all_levels_df(dfs[0], dfs[1], return_data=True)
end = time.time()
print("{:.2f} seconds".format(end-start))

120.91 seconds


In [19]:
# df_data.to_csv('test_year_trim_pmi_1980-1990.csv', index=False)

In [21]:
df

,pid,paper_rank,cl,cl_bottom,cl_top
218951,1,0.000000e+00,1:9:22:62:2,1:9:22:62,1
76831,2,0.000000e+00,1:2:8:87:3:2,1:2:8:87:3,1
212911,3,6.964010e-07,1:9:11:56:2,1:9:11:56,1
76742,5,7.958870e-07,1:2:8:78:1:3,1:2:8:78:1,1
130797,6,0.000000e+00,1:4:188:4:5,1:4:188:4,1
65849,46,3.410940e-07,1:2:5:23:24,1:2:5:23,1
136228,52,3.410940e-07,1:4:647:2,1:4:647,1
369333,57,0.000000e+00,1:24:315:3:2,1:24:315:3,1
942304,72,0.000000e+00,8912:2,8912,8912
67808,74,1.115120e-06,1:2:5:127:5,1:2:5:127,1


In [199]:
subset = df_data[df_data.pmi>0]
gb = subset.groupby('cl_left')

In [208]:
gbmax = gb.pmi.max()
gbmax[gbmax==1].count()

18052

In [211]:
gb2 = subset[subset.pmi==1].groupby('cl_left')
gb2.filter(lambda x: len(x)>1).sort_values('cl_left')

,x_idx,y_idx,pmi,level_left,level_right,cl_left,cl_right
9652,534,2343,1.0,4,3,1:10:14:31,1:14:287
9701,534,5728,1.0,4,4,1:10:14:31,1:14:287:1
106839,378,80642,1.0,5,4,1:10:2:72:2,4:6:43:3
42812,378,38745,1.0,5,5,1:10:2:72:2,4:6:43:3:1
10791,679,9614,1.0,5,5,1:114:2:2:4,1:29:1:55:1
23356,679,18027,1.0,5,4,1:114:2:2:4,1:29:1:55
45404,2192,12650,1.0,4,3,1:11:305:3,1:44:93
42859,2192,29969,1.0,4,4,1:11:305:3,1:44:93:1
42850,2846,29960,1.0,4,4,1:11:87:7,1:44:82:1
45395,2846,12643,1.0,4,3,1:11:87:7,1:44:82


In [226]:
df = dfs[1]
x = df[df.cl_bottom.str.startswith('1:14:287:1')]
x

,pid,paper_rank,cl,cl_bottom,cl_top
295673,523766,1.464810e-06,1:14:287:1:1,1:14:287:1,1
295675,1160092,1.705470e-07,1:14:287:1:3,1:14:287:1,1
295676,1160430,0.000000e+00,1:14:287:1:4,1:14:287:1,1
295674,3317235,8.299960e-07,1:14:287:1:2,1:14:287:1,1


In [227]:
df = dfs[0]
df[df.cl_bottom=='1:10:14:31']

,pid,paper_rank,cl,cl_bottom,cl_top
142170,523766,0.0,1:10:14:31:1,1:10:14:31,1


In [228]:
pidyear = pd.read_csv('pID_year.txt', index_col=0, header=None, squeeze=True)

In [229]:
x['year'] = x.pid.map(pidyear)
x

/home/jason/.local/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,pid,paper_rank,cl,cl_bottom,cl_top,year
295673,523766,1.464810e-06,1:14:287:1:1,1:14:287:1,1,1978.0
295675,1160092,1.705470e-07,1:14:287:1:3,1:14:287:1,1,1982.0
295676,1160430,0.000000e+00,1:14:287:1:4,1:14:287:1,1,1990.0
295674,3317235,8.299960e-07,1:14:287:1:2,1:14:287:1,1,1981.0


In [433]:
pd.DataFrame(pmis).sort_values('pmi', ascending=False)

,x_idx,y_idx,pmi
436001,55962,306230,1.000000
324627,86816,228614,1.000000
91021,205207,64183,1.000000
324601,45069,228593,1.000000
324609,66856,228598,1.000000
91016,218438,64180,1.000000
324614,66862,228603,1.000000
324617,66865,228606,1.000000
324618,66866,228607,1.000000
324665,66884,228638,1.000000


In [434]:

i=0
for x, y, v in np.sort(pmis, order='pmi')[::-1]:
    left_cat = get_category_name_from_code(left, x)
    right_cat = get_category_name_from_code(right, y)
    print(left_cat, right_cat, v)
    i+=1
    if i>=20:
        break

('9:5:3', '2:201:4', 1.0)
('9:5:2', '2:201:3', 1.0)
('9:1:6', '2:167:6', 1.0)
('9:1:5', '2:167:5', 1.0)
('9:1:4', '2:167:4', 1.0)
('99:6', '101:6', 1.0)
('99:4', '101:4', 1.0)
('99:3', '101:3', 1.0)
('996:3', '992:3', 1.0)
('996:2', '992:2', 1.0)
('992:5', '988:5', 1.0)
('992:3', '988:3', 1.0)
('992:2', '988:2', 1.0)
('992:1', '988:1', 1.0)
('987:2:1', '983:1:1', 1.0)
('986:2:1', '982:2:1', 1.0)
('983:3', '979:3', 1.0)
('97:4', '99:4', 1.0)
('97:3', '99:3', 1.0)
('972:1', '968:1', 1.0)


In [466]:
df_pmis[(df_pmis.level_left==df_pmis.level_right)&(df_pmis.pmi>0)].sort_values('pmi', ascending=False)

,x_idx,y_idx,pmi,level_left,level_right
0,0,0,inf,12,12
0,0,0,inf,11,11
25980,64536,24208,1.000000,5,5
25974,64528,24202,1.000000,5,5
25968,64526,24196,1.000000,5,5
25967,64525,24195,1.000000,5,5
25957,64514,24185,1.000000,5,5
25948,64505,24176,1.000000,5,5
25946,64504,24174,1.000000,5,5
25929,64549,24158,1.000000,5,5


In [467]:
df_pmis[(df_pmis.level_left!=df_pmis.level_right)&(df_pmis.pmi==1)].sort_values('pmi', ascending=False)

,x_idx,y_idx,pmi,level_left,level_right
10,96,10,1.0,2,3
183209,39509,142898,1.0,5,4
183311,12735,142982,1.0,5,4
183305,39600,142976,1.0,5,4
183304,39609,142975,1.0,5,4
183283,39612,142960,1.0,5,4
183269,39552,142950,1.0,5,4
183255,39549,142937,1.0,5,4
183222,39550,142911,1.0,5,4
183206,39506,142895,1.0,5,4


In [468]:
df_pmis[(df_pmis.level_left==df_pmis.level_right)&(df_pmis.pmi==1)].sort_values('pmi', ascending=False)

,x_idx,y_idx,pmi,level_left,level_right
34502,34455,34450,1.0,1,1
14636,55941,13357,1.0,5,5
14748,56216,13466,1.0,5,5
14743,56210,13461,1.0,5,5
14734,56199,13452,1.0,5,5
14733,56198,13451,1.0,5,5
14730,56195,13448,1.0,5,5
14715,56189,13434,1.0,5,5
14688,56174,13408,1.0,5,5
14687,56173,13407,1.0,5,5


In [469]:
df_pmis[df_pmis.x_idx==26791]

,x_idx,y_idx,pmi,level_left,level_right
26772,26791,26724,9.173829e-01,1,1
33882,26791,8141,1.449128e-01,1,2
94887,26791,64962,1.298092e-01,1,3
183838,26791,155415,-1.979733e-02,1,4
101416,26791,74391,-7.186238e-02,1,5
39398,26791,12587,-8.129154e-02,1,6
27867,26791,1076,-8.252207e-02,1,7
26862,26791,71,-8.261111e-02,1,8
26791,26791,0,-8.261713e-02,1,9
26791,26791,0,-8.261713e-02,1,10
